<a href="https://colab.research.google.com/github/JS0501/ESAA_OB/blob/main/ESAA0908.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **7.1 투표 기반 분류기**
#### 여러 분류기(로지스틱 회귀, SVM, 랜덤 포레스트...)를 갖고 있을 때 더 좋은 분류기를 만드는 매우 간단한 방법은 각 분류기의 예측을 모아 가장 많이 선택된 클래스를 예측하는 것.
#### 직접 투표 분류기: 다수결 투표로 정해지는 분류기
#### 다수결 투표 분류기가 앙상블에 포함된 개별 분류기 중 가장 뛰어난 것보다도 정확도가 높은 경우가 많음.
#### 각 분류기가 **약한 학습기**(랜덤 추측보다 조금 더 높은 성능을 내는 분류기)일지라도 충분하게 많고 다양하다면 앙상블은 (높은 정확도를 내는) **강한 학습기**가 될 수 있음.
#### 이는 **큰 수의 법칙** 때문임
#### ✅ 앙상블 방법은 **예측기가 가능한 한 서로 독립적일 때** 최고의 성능을 발휘. 다양한 분류기를 얻는 한 가지 방법은 각기 다른 알고리즘으로 학습시키는 것. 이렇게 하면 매우 다른 종류의 오차를 만들 가능성이 높기 때문에 앙상블 모델의 정확도를 향상시킴.

In [4]:
import warnings

warnings.filterwarnings('ignore')

# import package

import numpy as np

import os

# 5장에서의 moons dataset 불러오기

from sklearn.datasets import make_moons

from sklearn.model_selection import train_test_split

X,y = make_moons(n_samples=100, noise=0.15)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr',log_clf),('rf',rnd_clf),('svc',svm_clf)],voting='hard')
voting_clf.fit(X_train,y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [6]:
from sklearn.metrics import accuracy_score
for clf in (log_clf,rnd_clf,svm_clf,voting_clf):
  clf.fit(X_train,y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__,accuracy_score(y_test,y_pred))

LogisticRegression 0.7
RandomForestClassifier 1.0
SVC 0.9
VotingClassifier 0.9


#### 모든 분류기가 클래스의 확률을 예측할 수 있으면 (즉, predict_proba() 메서드가 있으면), 개별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스를 예측할 수 있음 -> **간접 투표**
#### 간접 투표 방식은 확률이 높은 투표에 비중을 더 두기 때문에 직접 투표 방식보다 성능이 높음
#### voting="hard" -> voting="soft" / 모든 분류기가 클래스의 확률을 추정할 수 있으면 됨

### **7.2 배깅과 페이스팅**
#### 다양한 분류기를 만드는 한 가지 방법은 각기 다른 훈련 알고리즘을 사용하는 것이었음.
#### 또 다른 방법은 같은 알고리즘을 사용하고 훈련 세트의 서브셋을 무작위로 구성하여 분류기를 각기 다르게 학습시키는 것.
#### **배깅**: 훈련 세트에서 중복을 허용하여 샘플링 (bootstrap aggregating)
#### **페이스팅**: 중복을 허용하지 않고 샘플링
#### 즉, 배깅과 페이스팅에서는 같은 훈련 샘플을 여러 개의 예측기에 걸쳐 사용할 수 있음. 하지만 배깅만이 한 예측기를 위해 같은 훈련 샘플을 여러 번 샘플링할 수 있음.
#### 모든 예측기가 훈련을 마치면 앙상블은 모든 예측기의 예측을 모아서 새로운 샘플에 대한 예측을 만듦.
#### 수집 함수를 전형적으로 분류일 때는 **통계적 최빈값**이고 회귀에대해서는 평균을 계산.
#### 개별 예측기는 원본 훈련 세트로 훈련시킨 것보다 훨씬 크게 편향되어 있지만 수집 함수를 통과하면 편향과 분산이 모두 감소함.
#### 일반적으로 앙상블의 결과는 원본 데이터셋으로 하나의 예측기를 훈련시킬 때와 비교해 편향은 비슷하지만 분산은 줄어듦.

### **7.2.1 사이킷런의 배깅과 페이스팅**

In [7]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=50, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train,y_train)
y_pred = bag_clf.predict(X_test)

### **7.2.2 oob 평가**
#### 배깅을 사용하면 어떤 샘플은 한 예측기를 위해 여러 번 샘플링되고 어떤 것은 전혀 선택되지 않을 수 있음. BaggingClassifier는 기본값으로 중복을 허용하여(bootstrap=True) 훈련 세트의 크기만큼인 m개 샘플을 선택함.
#### 이는 평균적으로 각 예측기에 훈련 샘플의 63% 정도만 샘플링된다는 것을 의미. 선택되지 않은 훈련 샘플의 나머지 37%를 oob(out-of-bag) 샘플이라고 부름. 예측기마다 남겨진 37%는 모두 다름.

In [9]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, n_jobs=-1, oob_score=True
)
bag_clf.fit(X_train,y_train)
bag_clf.oob_score_

0.925

In [10]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test,y_pred)

0.95

In [11]:
bag_clf.oob_decision_function_

array([[0.        , 1.        ],
       [0.06285714, 0.93714286],
       [0.        , 1.        ],
       [0.21428571, 0.78571429],
       [0.18579235, 0.81420765],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.14361702, 0.85638298],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.23076923, 0.76923077],
       [0.        , 1.        ],
       [0.15183246, 0.84816754],
       [0.03092784, 0.96907216],
       [0.68085106, 0.31914894],
       [0.98378378, 0.01621622],
       [0.08695652, 0.91304348],
       [0.91935484, 0.08064516],
       [0.16129032, 0.83870968],
       [0.08333333, 0.91666667],
       [0.1160221 , 0.8839779 ],
       [0.92783505, 0.07216495],
       [1.        , 0.        ],
       [0.64795918, 0.35204082],
       [1.        , 0.        ],
       [0.21118012, 0.78881988],
       [0.        , 1.        ],
       [0.97340426, 0.02659574],
       [0.

### **7.3 랜덤 패치와 랜덤 서브스페이스**
#### BaggingClassifier는 특성 샘플링도 지원.
#### 샘플링은 max_features, bootstrap_features 두 매개변수로 조절됨.

### **7.4 랜덤 포레스트**
#### 일반적으로 배깅 방법(또는 페이스팅)을 적용한 결정 트리의 앙상블.
#### 전형적으로 max_samples를 훈련 세트의 크기로 지정.
#### BaggingClassifier에 DecisionTreeClassifier를 넣어 만드는 대신 결정 트리에 최적화되어 사용하기 편리한 RandomForestClassifier를 사용할 수 있음.

In [12]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500,max_leaf_nodes=16,n_jobs=-1)
rnd_clf.fit(X_train,y_train)

y_pred_rf = rnd_clf.predict(X_test)

#### 랜덤 포레스트 알고리즘은 트리의 노드를 분할할 때 전체 특성 중에서 최선의 특성을 찾는 대신 무작위로 선택한 특성 후보 중에서 최적의 특성을 찾는 식으로 무작위성을 더 주입.
#### 이는 결국 트리를 더욱 다양하게 만들고 편향을 손해보는 대신 분산을 낮추어 전체적으로 더 훌륭한 모델을 만들어냄.

In [13]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features="auto",max_leaf_nodes=16),
    n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1
)

### **7.4.1 엑스트라 트리**
#### 랜덤 포레스트에서 트리를 만들 때 각 노드는 무작위로 특성의 서브셋을 만들어 분할에 사용.
#### 트리를 더욱 무작위하게 만들기 위해 최적의 임곗값을 찾는 대신 후보 특성을 사용해 무작위로 분할한 다음 그중에서 최상의 분할을 선택 -> **익스트림 랜덤 트리 앙상블**(또는 **엑스트라 트리**)
#### 여기서도 역시 편향이 늘어나지만 대신 분산을 낮추게 됨.
#### 모든 노드에서 특성마다 가장 최적의 임곗값을 찾는 것이 트리 알고리즘에서 가장 시간이 많이 소요되는 작업 중 하나이므로 일반적인 랜덤 포레스트보다 엑스트라 트리가 훨씬 빠름.

### **7.4.2 특성 중요도**
#### 랜덤 포레스트의 또 다른 장점은 특성의 상대적 중요도를 측정하기 쉽다는 것.
#### 사이킷런은 훈련이 끝난 뒤 특성마다 자동으로 이 점수를 계산하고 중요도의 전체합이 1이 되도록 결괏값을 정규화합니다. 이 값은 feature_importances_ 변수에 저장되어 있음.

In [15]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500,n_jobs=-1)
rnd_clf.fit(iris["data"],iris["target"])
for name, score in zip(iris["feature_names"],rnd_clf.feature_importances_):
  print(name,score)

sepal length (cm) 0.1059113505820875
sepal width (cm) 0.023665534917525797
petal length (cm) 0.43387672840061575
petal width (cm) 0.4365463860997709
